# 객체감지

In [4]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
result = model('sample.jpg')
result


image 1/1 c:\githome\wn7_dl_rep\sample.jpg: 384x640 1 cat, 1 bed, 36.2ms
Speed: 3.0ms preprocess, 36.2ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [26]:
# 바운딩박스, 라벨 출력
result = model('sample.jpg')
result[0].boxes


image 1/1 c:\githome\wn7_dl_rep\sample.jpg: 384x640 1 cat, 1 bed, 18.8ms
Speed: 3.7ms preprocess, 18.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([15., 59.], device='cuda:0')
conf: tensor([0.9199, 0.3141], device='cuda:0')
data: tensor([[1.3989e+02, 1.6935e+01, 8.4657e+02, 4.5707e+02, 9.1994e-01, 1.5000e+01],
        [1.4731e+01, 9.3849e-01, 8.4567e+02, 4.6657e+02, 3.1414e-01, 5.9000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (478, 849)
shape: torch.Size([2, 6])
xywh: tensor([[493.2305, 237.0033, 706.6719, 440.1364],
        [430.2020, 233.7550, 830.9418, 465.6331]], device='cuda:0')
xywhn: tensor([[0.5810, 0.4958, 0.8324, 0.9208],
        [0.5067, 0.4890, 0.9787, 0.9741]], device='cuda:0')
xyxy: tensor([[139.8945,  16.9351, 846.5665, 457.0715],
        [ 14.7311,   0.9385, 845.6729, 466.5715]], device='cuda:0')
xyxyn: tensor([[0.1648, 0.0354, 0.9971, 0.9562],
        [0.0174, 0.0020, 0.9961, 0.9761]], device='cuda:0')

In [27]:
import cv2
img = cv2.imread('sample.jpg')

In [29]:


for box in result[0].boxes:
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    #신뢰도점수
    conf = float(box.conf[0])
    
    #바운딩 박스
    cords = box.xyxy[0].tolist()
    x1,y1,x2,y2 = map(int, cords)
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 2)
    cv2.putText(img, f'{class_name} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))


cv2.imshow('yolov8 object detection', img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', img)

True

# segmentation

In [55]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')
result = model('sample.jpg')
result[0].show()


image 1/1 c:\githome\wn7_dl_rep\sample.jpg: 384x640 1 cat, 24.4ms
Speed: 3.3ms preprocess, 24.4ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
import numpy as np

img_new = cv2.imread('sample.jpg')
overlay = img_new.copy() #마스크용

for box, mask in zip(result[0].boxes, result[0].masks):
    polygon = mask.xy[0].astype(np.int32)
    cv2.fillPoly(overlay, [polygon], (0,255,0))
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    #신뢰도점수
    conf = float(box.conf[0])
    
    #바운딩 박스
    cords = box.xyxy[0].tolist()
    x1,y1,x2,y2 = map(int, cords)
    cv2.rectangle(img_new, (x1,y1), (x2,y2), (0,0,255), 2)
    cv2.putText(img_new, f'{class_name} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))

#img_new + overlay 겹쳐서 출력
alpha = 0.5
final_img = cv2.addWeighted(overlay,alpha, img_new,1-alpha, 0)

#최종출력
cv2.imshow('yolov8 object segmention', final_img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', final_img)

True

# 분류 (생략)
# 자세

In [58]:
pose_model = YOLO('yolov8n-pose.pt')
pose_result = pose_model('sample1.jpg')
image = cv2.imread('sample1.jpg')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x,y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(image,(int(x), int(y)), 3, (0,0,255), -1)
            
cv2.imshow('yolov8 pose estimation', image)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('pose_result.jpg', image)
            


image 1/1 c:\githome\wn7_dl_rep\sample1.jpg: 640x480 3 persons, 27.3ms
Speed: 2.5ms preprocess, 27.3ms inference, 6.5ms postprocess per image at shape (1, 3, 640, 480)


True